In [1]:
import argparse
import numpy as np
from packages.vocab import Vocab
from packages.batch import Batch
from model import Model
from packages.functions import to_cuda, num_to_var
import torch
from torch import nn,optim
import os
import random
from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as pad

parser = argparse.ArgumentParser()

In [2]:
class Args(object):
    data_dir='/home/jatin/intern/internenv/cnn/2.stories_tokenized_100'
    word2idx='word2idx.npy'
    idx2word='idx2word.npy'
    max_enc=200
    max_dec=50
    min_dec=35
    vocab_size=500
    max_oovs = 20
    train = True
    epochs = 20
    load_model = ''
    hidden_size = 256
    embed_size = 128
    lr = 0.15
    cov_lambda = 1.0
    beam = 4
    cuda = True
args = Args()

In [3]:
# def main(args):
# obtain vocabulary
vocab = Vocab(args.vocab_size)
vocab.w2i = np.load(args.word2idx).item()
vocab.i2w = np.load(args.idx2word).item()
vocab.count = len(vocab.w2i)

# obtain dataset in batches
file_list = os.listdir(args.data_dir)
batch = Batch(file_list, args.max_enc, args.max_dec, args.max_oovs)

# load model
if args.load_model != '':
    model = torch.load(args.load_model)
else:
    model = Model(args)
model = to_cuda(model)

# get loss and optimizers
opt = optim.Adam(params=model.parameters(),lr=args.lr)
criterion = nn.NLLLoss()

# computation for each epoch
epoch = 0
while (epoch<args.epochs):
    epoch+=1
    random.shuffle(file_list)
    for file in file_list[0:1]:
        opt.zero_grad()
        with open(os.path.join(args.data_dir,file)) as f:
            minibatch = f.read()
        stories,summaries = batch.process_minibatch(minibatch,vocab)
        out_list, cov_loss = model(stories, summaries, batch, vocab, True)
        
        # get packed versions
        target = num_to_var(summaries[:,1:])
        target = pack(target, batch.output_lens.to_list(),batch_first=True)[0]
        pad_out = pack(out_list, batch.output_lens.to_list(),batch_first=True)[0]
        pad_out = torch.log(pad_out)
        loss = criterion(pad_out,target)+cov_loss
        loss.backward()
        opt.step()
        print("got thru batch!")
# if __name__ == "__main__":
#     main(args)

/home/jatin/.local/lib/python3.5/site-packages/torch/cuda/__init__.py:116: UserWarning: 
    Found GPU0 GeForce 820M which is of cuda capability 2.1.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


RuntimeError: CUDNN_STATUS_ARCH_MISMATCH

In [4]:
np.zeros([10,40],dtype=float)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   

In [5]:
# for selective masking while preserving sizes
from collections import Counter 
import time

In [6]:
start = time.time()
numbers = stories.reshape(-1).tolist()
set_numbers = list(set(numbers))
c = Counter(numbers)
dup_list = [k for k in set_numbers if c[k] > 1]
print("Number of iters: %d" % len(dup_list))
elapsed = time.time()
print("Time elapsed: ", elapsed-start)

NameError: name 'stories' is not defined

In [8]:
for dup in dup_list[0:1]:
    mask = np.array(stories==dup,dtype=float)

NameError: name 'dup_list' is not defined

In [9]:
attn = np.random.randn(mask.shape[0],mask.shape[1])

NameError: name 'mask' is not defined

In [10]:
M = mask*attn.sum(1).reshape(-1,1)

NameError: name 'mask' is not defined

In [11]:
from torch.autograd import Variable
mask2 = Variable(torch.Tensor(mask).cuda())
attn2 = Variable(torch.Tensor(attn).cuda())

NameError: name 'mask' is not defined

In [12]:
M2=torch.mul(mask2, attn2.sum(1).unsqueeze(1))

NameError: name 'mask2' is not defined

In [13]:
attn2[-1]

NameError: name 'attn2' is not defined

In [14]:
import argparse

In [15]:
# hyperparameters
hidden_dim = 256
embed_dim = 128
batch_size = 16
num_samples = 92579
max_encoder_steps = 400
max_decoder_steps = 100
beam_size = 4
min_decoder_steps = 35 # min size of generated sequence
vocab_size = 500
lr = 0.15
adagrad_init_acc = 0.1 # deprecated for pytorch
rand_unif_init_mag = 0.02 # magnitude for lstm cells during random init
trunc_norm_init_std = 1e-4 # std of truncated norm initialization
max_grad_norm = 2.0 # so they do apply gradient clipping
max_oovs = 20 # maximum number of oovs allowed?
coverage_loss = 1.0 # lambda

In [16]:
# get vocabulary
vocab = Vocab(500)
vocab.w2i = np.load('word2idx.npy').item()
vocab.i2w = np.load('idx2word.npy').item()
vocab.count = len(vocab.w2i)

In [26]:
# get dataset in batches
file_dir = '/home/jatin/intern/internenv/cnn/stories_merged_100/'
file_list = os.listdir(file_dir)
batch = Batch(file_list,400,100,50)

In [27]:
batch.init_minibatch()
with open(os.path.join(file_dir,file_list[20])) as f:
    minibatch = f.read()
    minibatch = minibatch.split('\n\n')
    minibatch = [line for line in minibatch if not line.startswith(":==:")]
stories, summaries = batch.process_minibatch(minibatch,vocab)

ValueError: not enough values to unpack (expected 2, got 1)

In [14]:
stories[idx]

NameError: name 'idx' is not defined

In [15]:
idx=22
' '.join(vocab.idx_list_to_word_list(stories[idx]))

IndexError: index 22 is out of bounds for axis 0 with size 10

In [16]:
batch.idx2oov_list

[{501: 'madrid',
  502: 'spain',
  503: 'car',
  504: 'bomb',
  505: 'exploded',
  506: 'morning',
  507: 'northern',
  508: 'basque',
  509: 'region',
  510: 'killing',
  511: 'officer',
  512: 'inside',
  513: 'vehicle',
  514: 'investigate',
  515: 'destroyed',
  516: 'prime',
  517: 'minister',
  518: 'blamed',
  519: 'attack',
  520: 'separatist'},
 {501: 'families',
  502: 'started',
  503: 'karen',
  504: 'suffern',
  505: 'planning',
  506: 'christmas',
  507: 'financially',
  508: 'struggling',
  509: 'single',
  510: 'mother',
  511: 'fraternal',
  512: 'twins',
  513: 'recently',
  514: '8-year',
  515: 'olds',
  516: 'write',
  517: 'list',
  518: 'santa',
  519: 'works',
  520: 'administrative'},
 {501: 'saga',
  502: 'trayvon',
  503: 'martin',
  504: 'continued',
  505: 'energize',
  506: 'thousands',
  507: 'spurring',
  508: 'demonstrations',
  509: 'turning',
  510: 'regularly',
  511: 'scheduled',
  512: 'commission',
  513: 'meeting',
  514: 'hot',
  515: 'ticket',


In [17]:
' '.join(vocab.idx_list_to_word_list(stories[idx],batch.idx2oov_list[idx]))

IndexError: index 22 is out of bounds for axis 0 with size 10

In [18]:
unked = batch.unk_minibatch(stories[idx],vocab)
' '.join(vocab.idx_list_to_word_list(unked,batch.idx2oov_list[idx]))

IndexError: index 22 is out of bounds for axis 0 with size 10

In [19]:
' '.join(vocab.idx_list_to_word_list(summaries[idx],batch.idx2oov_list[idx]))

IndexError: index 22 is out of bounds for axis 0 with size 10

In [20]:
import torch.nn as nn
help(nn.LSTMCell)

Help on class LSTMCell in module torch.nn.modules.rnn:

class LSTMCell(RNNCellBase)
 |  A long short-term memory (LSTM) cell.
 |  
 |  .. math::
 |  
 |      \begin{array}{ll}
 |      i = \sigma(W_{ii} x + b_{ii} + W_{hi} h + b_{hi}) \\
 |      f = \sigma(W_{if} x + b_{if} + W_{hf} h + b_{hf}) \\
 |      g = \tanh(W_{ig} x + b_{ig} + W_{hc} h + b_{hg}) \\
 |      o = \sigma(W_{io} x + b_{io} + W_{ho} h + b_{ho}) \\
 |      c' = f * c + i * g \\
 |      h' = o \tanh(c') \\
 |      \end{array}
 |  
 |  where :math:`\sigma` is the sigmoid function.
 |  
 |  Args:
 |      input_size: The number of expected features in the input `x`
 |      hidden_size: The number of features in the hidden state `h`
 |      bias: If `False`, then the layer does not use bias weights `b_ih` and
 |          `b_hh`. Default: ``True``
 |  
 |  Inputs: input, (h_0, c_0)
 |      - **input** of shape `(batch, input_size)`: tensor containing input features
 |      - **h_0** of shape `(batch, hidden_size)`: tensor co

In [21]:
batch.oov2idx_list[idx]

IndexError: list index out of range

In [22]:
np.ones([10])

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])